In [0]:
%run ../Includes/Copy-Datasets

In [0]:
CREATE TABLE orders AS
SELECT * FROM parquet.`${dataset.bookstore}/orders`;

In [0]:
SELECT * FROM orders;

# Overwrite table

In [0]:
CREATE OR REPLACE TABLE orders AS
SELECT * FROM parquet.`${dataset.bookstore}/orders`;

In [0]:
DESC HISTORY orders;

In [0]:
-- It can only overwrite an existing table
-- It can only overwrite the new records that match the current table schema.
INSERT OVERWRITE orders SELECT * FROM parquet.`${dataset.bookstore}/orders`;

In [0]:
DESC HISTORY orders;

In [0]:
-- Error
INSERT OVERWRITE orders SELECT *, current_timestamp() FROM parquet.`${dataset.bookstore}/orders`;

In [0]:
-- It can insert duplicates data if you execute again
INSERT INTO orders SELECT * FROM parquet.`${dataset.bookstore}/orders-new`;

In [0]:
SELECT COUNT(*) FROM orders;

In [0]:
CREATE OR REPLACE TEMP VIEW customers_updates AS
SELECT * FROM json.`${dataset.bookstore}/customers-json-new`;
MERGE INTO customers c
USING customers_updates u
ON c.customer_id = u.customer_id
WHEN MATCHED AND c.email IS NULL AND u.email IS NOT NULL THEN
  UPDATE SET c.email = u.email
WHEN NOT MATCHED THEN INSERT *

In [0]:
CREATE OR REPLACE TEMP VIEW book_updates
  (book_id STRING, title STRING, author STRING, category STRING, price DOUBLE)
USING csv
OPTIONS (
  path "${dataset.bookstore}/books-csv-new",
  header = "true",
  delimiter = ";"
);
SELECT * FROM book_updates;

In [0]:
MERGE INTO books b
USING book_updates u
ON b.book_id = u.book_id AND b.title = u.title
WHEN NOT MATCHED AND u.category = "Computer Science" THEN
  INSERT *

In [0]:
-- Schema Evolution for Merge
/*
In Databricks Runtime 15.4 LTS and above, you can enable schema evolution in a merge statement using the WITH SCHEMA EVOLUTION clause:
This syntax allows the schema of the target table to automatically update when new columns are present in the source. In this case, If customers_updates includes additional fields not present in the customers table, the schema of the table will evolve to include these fields, and their values will be inserted/updated accordingly during the merge operation.
*/
MERGE WITH SCHEMA EVOLUTION INTO customers c
USING customers_updates u
ON c.customer_id = u.customer_id
WHEN MATCHED AND c.email IS NULL AND u.email IS NOT NULL THEN
  UPDATE SET email = u.email, updated = u.updated
WHEN NOT MATCHED THEN INSERT *